In [1]:
# importing the required libraries
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [ ]:
for i in range(1,1000,10):
    # importing the training data from the training file already present in EMNIST dataset
    training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = i+1, shuffle = True)
# importing the testing data
    testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)
    # creating the neural network class. this class will consist of 1 convolutional layer, 1 max pooling layer and 3 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
    class Net(nn.Module):
        def __init__(self):
            super(Net,self).__init__()
#         input is 1X28X28, output is 26 letters
            self.conv1 = nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
            self.fc1 = nn.Linear(18 * 14 * 14, 256)
            self.fc2 = nn.Linear(256, 64)
            self.fc3 = nn.Linear(64, 27)
        
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool(x)
            x = x.view(-1, 18 * 14 *14)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(x)
        # defining the relationship between input and output sizes for usage afterwards
    def outputsize(in_size,kernel_size,stride,padding):
        output = int((in_size-kernel_size+2*(padding)) / stride) + 1
        return output
# defining the various hyperparameters
    epochs = 3 
    log_interval = 25
    network = Net()
    optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.5)
# creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
    train_losses = []
    train_counter = []
    test_losses = []
    test_counter = [i*len(training.dataset) for i in range(epochs+1)]
    # defining the training network
    def train(epoch):
        network.train()
        for batch_idx, (data,target) in enumerate(training):
#       making all the gradients zero before the start of each epoch
            optimizer.zero_grad()
#     input output and backpropagation steps along with the application of NLL loss function for now
            output = network(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
#         printing the results after each mini batch has been trained during a particular epoch
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
                train_losses.append(loss.item())
                train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
                torch.save(network.state_dict(), '/results/model.pth')
                torch.save(optimizer.state_dict(), '/results/optimizer.pth')
# defining the testing network
    def test():
        network.eval()
        test_loss = 0
        correct = 0
        losstest = []
#   calculating the loss for each sample and the accuracy
        with torch.no_grad():
            for data, target in testing:
                output = network(data)
                test_loss += F.nll_loss(output, target, size_average = False).item()
                pred = output.data.max(1, keepdim = True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()
#         initialising the various parameters to calculate accuracy and print it
            test_loss /= len(testing.dataset)
            losstest.append(test_loss)
            print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
# initializing the entire network with the datasets and using the above functions to calculate the accuracy
# first checking the accuracy with random guesses(network uninitialised)
    test()
    for epoch in range(1,epochs+1):
        train(epoch)
        test()        

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\admin\anaconda3\lib\site-packages\torch\nn\_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. Loss: 3.2973, Accuracy: 761/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.264349
Train Epoch: 1 [50/124800 (0%)]	Loss: 3.421684
Train Epoch: 1 [100/124800 (0%)]	Loss: 3.616001
Train Epoch: 1 [150/124800 (0%)]	Loss: 3.113719
Train Epoch: 1 [200/124800 (0%)]	Loss: 3.243359
Train Epoch: 1 [250/124800 (0%)]	Loss: 2.624514
Train Epoch: 1 [300/124800 (0%)]	Loss: 1.857269
Train Epoch: 1 [350/124800 (0%)]	Loss: 3.549759
Train Epoch: 1 [400/124800 (0%)]	Loss: 3.620430
Train Epoch: 1 [450/124800 (0%)]	Loss: 2.078073
Train Epoch: 1 [500/124800 (0%)]	Loss: 2.670196
Train Epoch: 1 [550/124800 (0%)]	Loss: 2.071370
Train Epoch: 1 [600/124800 (0%)]	Loss: 2.127208
Train Epoch: 1 [650/124800 (1%)]	Loss: 2.890085
Train Epoch: 1 [700/124800 (1%)]	Loss: 2.218265
Train Epoch: 1 [750/124800 (1%)]	Loss: 3.672971
Train Epoch: 1 [800/124800 (1%)]	Loss: 1.760727
Train Epoch: 1 [850/124800 (1%)]	Loss: 1.448982
Train Epoch: 1 [900/124800 (1%)]	Loss: 3.583396
Train Epoch: 1 [950/124800 (1%)]	Lo

Train Epoch: 1 [8300/124800 (7%)]	Loss: 0.172423
Train Epoch: 1 [8350/124800 (7%)]	Loss: 0.994543
Train Epoch: 1 [8400/124800 (7%)]	Loss: 0.465389
Train Epoch: 1 [8450/124800 (7%)]	Loss: 6.352038
Train Epoch: 1 [8500/124800 (7%)]	Loss: 0.950920
Train Epoch: 1 [8550/124800 (7%)]	Loss: 1.235012
Train Epoch: 1 [8600/124800 (7%)]	Loss: 0.082320
Train Epoch: 1 [8650/124800 (7%)]	Loss: 0.349003
Train Epoch: 1 [8700/124800 (7%)]	Loss: 1.370856
Train Epoch: 1 [8750/124800 (7%)]	Loss: 0.220369
Train Epoch: 1 [8800/124800 (7%)]	Loss: 0.015625
Train Epoch: 1 [8850/124800 (7%)]	Loss: 0.243539
Train Epoch: 1 [8900/124800 (7%)]	Loss: 1.172817
Train Epoch: 1 [8950/124800 (7%)]	Loss: 2.240778
Train Epoch: 1 [9000/124800 (7%)]	Loss: 0.300250
Train Epoch: 1 [9050/124800 (7%)]	Loss: 0.195287
Train Epoch: 1 [9100/124800 (7%)]	Loss: 0.915416
Train Epoch: 1 [9150/124800 (7%)]	Loss: 0.637512
Train Epoch: 1 [9200/124800 (7%)]	Loss: 0.735058
Train Epoch: 1 [9250/124800 (7%)]	Loss: 0.500760
Train Epoch: 1 [9300

Train Epoch: 1 [16400/124800 (13%)]	Loss: 0.401680
Train Epoch: 1 [16450/124800 (13%)]	Loss: 0.202709
Train Epoch: 1 [16500/124800 (13%)]	Loss: 0.243363
Train Epoch: 1 [16550/124800 (13%)]	Loss: 0.154744
Train Epoch: 1 [16600/124800 (13%)]	Loss: 0.665646
Train Epoch: 1 [16650/124800 (13%)]	Loss: 1.053104
Train Epoch: 1 [16700/124800 (13%)]	Loss: 0.026061
Train Epoch: 1 [16750/124800 (13%)]	Loss: 0.306279
Train Epoch: 1 [16800/124800 (13%)]	Loss: 1.575804
Train Epoch: 1 [16850/124800 (14%)]	Loss: 0.073986
Train Epoch: 1 [16900/124800 (14%)]	Loss: 0.741214
Train Epoch: 1 [16950/124800 (14%)]	Loss: 0.944503
Train Epoch: 1 [17000/124800 (14%)]	Loss: 0.430709
Train Epoch: 1 [17050/124800 (14%)]	Loss: 0.259729
Train Epoch: 1 [17100/124800 (14%)]	Loss: 1.007753
Train Epoch: 1 [17150/124800 (14%)]	Loss: 0.712775
Train Epoch: 1 [17200/124800 (14%)]	Loss: 0.065171
Train Epoch: 1 [17250/124800 (14%)]	Loss: 0.165788
Train Epoch: 1 [17300/124800 (14%)]	Loss: 0.240535
Train Epoch: 1 [17350/124800 (1

Train Epoch: 1 [24400/124800 (20%)]	Loss: 0.032393
Train Epoch: 1 [24450/124800 (20%)]	Loss: 0.279923
Train Epoch: 1 [24500/124800 (20%)]	Loss: 0.536250
Train Epoch: 1 [24550/124800 (20%)]	Loss: 0.363880
Train Epoch: 1 [24600/124800 (20%)]	Loss: 0.388005
Train Epoch: 1 [24650/124800 (20%)]	Loss: 0.138968
Train Epoch: 1 [24700/124800 (20%)]	Loss: 0.002986
Train Epoch: 1 [24750/124800 (20%)]	Loss: 0.004293
Train Epoch: 1 [24800/124800 (20%)]	Loss: 0.003101
Train Epoch: 1 [24850/124800 (20%)]	Loss: 0.007340
Train Epoch: 1 [24900/124800 (20%)]	Loss: 0.032294
Train Epoch: 1 [24950/124800 (20%)]	Loss: 0.440722
Train Epoch: 1 [25000/124800 (20%)]	Loss: 0.049913
Train Epoch: 1 [25050/124800 (20%)]	Loss: 0.027272
Train Epoch: 1 [25100/124800 (20%)]	Loss: 0.316365
Train Epoch: 1 [25150/124800 (20%)]	Loss: 0.190638
Train Epoch: 1 [25200/124800 (20%)]	Loss: 0.001918
Train Epoch: 1 [25250/124800 (20%)]	Loss: 0.000113
Train Epoch: 1 [25300/124800 (20%)]	Loss: 1.381296
Train Epoch: 1 [25350/124800 (2

...

Train Epoch: 3 [63900/124800 (51%)]	Loss: 0.432716
Train Epoch: 3 [67450/124800 (54%)]	Loss: 0.453060
Train Epoch: 3 [71000/124800 (57%)]	Loss: 0.411084
Train Epoch: 3 [74550/124800 (60%)]	Loss: 0.468120
Train Epoch: 3 [78100/124800 (63%)]	Loss: 0.344167
Train Epoch: 3 [81650/124800 (65%)]	Loss: 0.379710
Train Epoch: 3 [85200/124800 (68%)]	Loss: 0.422809
Train Epoch: 3 [88750/124800 (71%)]	Loss: 0.313764
Train Epoch: 3 [92300/124800 (74%)]	Loss: 0.483560
Train Epoch: 3 [95850/124800 (77%)]	Loss: 0.407589
Train Epoch: 3 [99400/124800 (80%)]	Loss: 0.306813
Train Epoch: 3 [102950/124800 (82%)]	Loss: 0.284307
Train Epoch: 3 [106500/124800 (85%)]	Loss: 0.393193
Train Epoch: 3 [110050/124800 (88%)]	Loss: 0.427099
Train Epoch: 3 [113600/124800 (91%)]	Loss: 0.405140
Train Epoch: 3 [117150/124800 (94%)]	Loss: 0.435722
Train Epoch: 3 [120700/124800 (97%)]	Loss: 0.495789
Train Epoch: 3 [124250/124800 (100%)]	Loss: 0.402534

Test set: Avg. Loss: 0.3909, Accuracy: 18317/20800 (88%)


Test set: Avg.

Train Epoch: 2 [16200/124800 (13%)]	Loss: 0.722022
Train Epoch: 2 [20250/124800 (16%)]	Loss: 0.639614
Train Epoch: 2 [24300/124800 (19%)]	Loss: 0.669920
Train Epoch: 2 [28350/124800 (23%)]	Loss: 0.689645
Train Epoch: 2 [32400/124800 (26%)]	Loss: 0.655032
Train Epoch: 2 [36450/124800 (29%)]	Loss: 0.592830
Train Epoch: 2 [40500/124800 (32%)]	Loss: 0.672087
Train Epoch: 2 [44550/124800 (36%)]	Loss: 0.741492
Train Epoch: 2 [48600/124800 (39%)]	Loss: 0.602588
Train Epoch: 2 [52650/124800 (42%)]	Loss: 0.610145
Train Epoch: 2 [56700/124800 (45%)]	Loss: 0.666770
Train Epoch: 2 [60750/124800 (49%)]	Loss: 0.668102
Train Epoch: 2 [64800/124800 (52%)]	Loss: 0.582959
Train Epoch: 2 [68850/124800 (55%)]	Loss: 0.609928
Train Epoch: 2 [72900/124800 (58%)]	Loss: 0.715921
Train Epoch: 2 [76950/124800 (62%)]	Loss: 0.569656
Train Epoch: 2 [81000/124800 (65%)]	Loss: 0.486086
Train Epoch: 2 [85050/124800 (68%)]	Loss: 0.548838
Train Epoch: 2 [89100/124800 (71%)]	Loss: 0.664756
Train Epoch: 2 [93150/124800 (7

Train Epoch: 1 [18200/124800 (15%)]	Loss: 2.684134
Train Epoch: 1 [22750/124800 (18%)]	Loss: 2.245968
Train Epoch: 1 [27300/124800 (22%)]	Loss: 1.748984
Train Epoch: 1 [31850/124800 (26%)]	Loss: 1.550877
Train Epoch: 1 [36400/124800 (29%)]	Loss: 1.489723
Train Epoch: 1 [40950/124800 (33%)]	Loss: 1.475025
Train Epoch: 1 [45500/124800 (36%)]	Loss: 1.332725
Train Epoch: 1 [50050/124800 (40%)]	Loss: 1.140283
Train Epoch: 1 [54600/124800 (44%)]	Loss: 1.157574
Train Epoch: 1 [59150/124800 (47%)]	Loss: 1.173919
Train Epoch: 1 [63700/124800 (51%)]	Loss: 0.996301
Train Epoch: 1 [68250/124800 (55%)]	Loss: 1.141329
Train Epoch: 1 [72800/124800 (58%)]	Loss: 1.100403
Train Epoch: 1 [77350/124800 (62%)]	Loss: 1.180174
Train Epoch: 1 [81900/124800 (66%)]	Loss: 1.256505
Train Epoch: 1 [86450/124800 (69%)]	Loss: 1.077888
Train Epoch: 1 [91000/124800 (73%)]	Loss: 0.911555
Train Epoch: 1 [95550/124800 (77%)]	Loss: 0.817419
Train Epoch: 1 [100100/124800 (80%)]	Loss: 0.818463
Train Epoch: 1 [104650/124800 

Train Epoch: 3 [105600/124800 (85%)]	Loss: 0.485882
Train Epoch: 3 [110400/124800 (88%)]	Loss: 0.467322
Train Epoch: 3 [115200/124800 (92%)]	Loss: 0.557392
Train Epoch: 3 [120000/124800 (96%)]	Loss: 0.374800

Test set: Avg. Loss: 0.4674, Accuracy: 17859/20800 (85%)


Test set: Avg. Loss: 3.3024, Accuracy: 746/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.309535
Train Epoch: 1 [5050/124800 (4%)]	Loss: 3.200760
Train Epoch: 1 [10100/124800 (8%)]	Loss: 3.079114
Train Epoch: 1 [15150/124800 (12%)]	Loss: 2.802871
Train Epoch: 1 [20200/124800 (16%)]	Loss: 2.467294
Train Epoch: 1 [25250/124800 (20%)]	Loss: 2.112524
Train Epoch: 1 [30300/124800 (24%)]	Loss: 1.671586
Train Epoch: 1 [35350/124800 (28%)]	Loss: 1.402218
Train Epoch: 1 [40400/124800 (32%)]	Loss: 1.343676
Train Epoch: 1 [45450/124800 (36%)]	Loss: 1.258219
Train Epoch: 1 [50500/124800 (40%)]	Loss: 1.384466
Train Epoch: 1 [55550/124800 (44%)]	Loss: 1.272406
Train Epoch: 1 [60600/124800 (49%)]	Loss: 1.112534
Train Epoch: 1 [65650/



Test set: Avg. Loss: 3.2968, Accuracy: 1040/20800 (5%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.295281
Train Epoch: 1 [5550/124800 (4%)]	Loss: 3.260293
Train Epoch: 1 [11100/124800 (9%)]	Loss: 3.184111
Train Epoch: 1 [16650/124800 (13%)]	Loss: 3.056833
Train Epoch: 1 [22200/124800 (18%)]	Loss: 2.840955
Train Epoch: 1 [27750/124800 (22%)]	Loss: 2.357121
Train Epoch: 1 [33300/124800 (27%)]	Loss: 2.006554
Train Epoch: 1 [38850/124800 (31%)]	Loss: 1.581647
Train Epoch: 1 [44400/124800 (36%)]	Loss: 1.442755
Train Epoch: 1 [49950/124800 (40%)]	Loss: 1.347907
Train Epoch: 1 [55500/124800 (44%)]	Loss: 1.218028
Train Epoch: 1 [61050/124800 (49%)]	Loss: 1.332050
Train Epoch: 1 [66600/124800 (53%)]	Loss: 1.250250
Train Epoch: 1 [72150/124800 (58%)]	Loss: 1.139743
Train Epoch: 1 [77700/124800 (62%)]	Loss: 1.094911
Train Epoch: 1 [83250/124800 (67%)]	Loss: 1.202462
Train Epoch: 1 [88800/124800 (71%)]	Loss: 1.045611
Train Epoch: 1 [94350/124800 (75%)]	Loss: 1.015886
Train Epoch: 1 [99900/124800 (80